In [216]:
#import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
#download provided NFL data
#assumes that all files are in the same folder as this file
df_games = pd.read_csv('games.csv')
df_player_play = pd.read_csv('player_play.csv')
df_players = pd.read_csv('players.csv')
df_plays = pd.read_csv('plays.csv')
df_w1 = pd.read_csv('tracking_week_1.csv')
df_w2 = pd.read_csv('tracking_week_2.csv')
df_w3 = pd.read_csv('tracking_week_3.csv')
df_w4 = pd.read_csv('tracking_week_4.csv')
df_w5 = pd.read_csv('tracking_week_5.csv')
df_w6 = pd.read_csv('tracking_week_6.csv')
df_w7 = pd.read_csv('tracking_week_7.csv')
df_w8 = pd.read_csv('tracking_week_8.csv')
df_w9 = pd.read_csv('tracking_week_9.csv')

Notes on DataFrames
- shiftSinceLineset (may be useful)
- event (lineset?)

In [33]:
#concatenate the player tracking dataframes
tracking = pd.concat([df_w1,df_w2,df_w3,df_w4,df_w5,df_w6,df_w7,df_w8,df_w9])

In [34]:
#change all plays to going in the same direction
#to save space, also multiply by 100 and convert to int

def standardize_df(df):
    #set all plays go left to right, and convert floats to ints to save memory
    df['x_std'] = np.where(
        df["playDirection"] == "left",
        (120 - df["x"].to_numpy()) * 100,
        df["x"].to_numpy() * 100
    ).astype(int)
    df['y_std'] = np.where(
        df["playDirection"] == "left",
        (120 - df["y"].to_numpy()) * 100,
        df["y"].to_numpy() * 100
    ).astype(int)

    #standardize and clean the 'o' column to go left to right
    df["o_std"] = (-(df["o"] - 90)) % 360
    df["o_std"] = np.where(
        df["playDirection"] == "left",
        (180 - df["o_std"]) % 360,
        df["o_std"]
    )
    df["o_std"] = (df["o_std"] * 100).fillna(0).astype(int)  #scale and convert to int
    
    #standardize and clean the 'dir' column to go left to right
    df["dir_std"] = (-(df["dir"] - 90)) % 360
    df["dir_std"] = np.where(
        df["playDirection"] == "left",
        (180 - df["dir_std"]) % 360, 
        df["dir_std"]
    )
    df["dir_std"] = (df["dir_std"] * 100).fillna(0).astype(int)  #scale and convert to int

    #convert other cells to int
    df['s_std'] = (df['s']*100).astype(int)
    df['a_std'] = (df['a']*100).astype(int)
    df['dis_std'] = (df['dis']*100).astype(int)

    #downcast columns to save memory
    df['x_std'] = pd.to_numeric(df['x_std'], downcast='integer')
    df['y_std'] = pd.to_numeric(df['y_std'], downcast='integer')
    df['o_std'] = pd.to_numeric(df['o_std'], downcast='integer')
    df['dis_std'] = pd.to_numeric(df['dis_std'], downcast='integer')
    df['dir_std'] = pd.to_numeric(df['dir_std'], downcast='integer')
    df['s_std'] = pd.to_numeric(df['s_std'], downcast='integer')
    df['a_std'] = pd.to_numeric(df['a_std'], downcast='integer')


In [35]:
standardize_df(tracking)

In [36]:
#calculate run-dropback percentages in provided data
run_pct = len(df_plays[df_plays['passResult'].isnull()]) / len(df_plays)
dropback_pct = len(df_plays[df_plays['passResult'].notnull()]) / len(df_plays)
print(f"design run: {run_pct:.1%} \ndropback:   {dropback_pct:.1%}")

design run: 39.6% 
dropback:   60.4%


Rough Code I left in

In [40]:
tracking_unique_ids = tracking[['gameId', 'playId', 'nflId']].drop_duplicates()

In [41]:
len(tracking_unique_ids)

370852

In [104]:
#find which plays have the lineset for the play noted
lineset_unique_ids = tracking[tracking['event'] == 'line_set'][['gameId', 'playId', 'nflId']].drop_duplicates()

In [106]:
df_all = tracking_unique_ids.merge(lineset_unique_ids.drop_duplicates(), on=['gameId','playId','nflId'], 
                   how='left', indicator=True)
df_all = df_all[df_all['_merge'] == 'left_only']

In [108]:
# #start with plays that have lineset noted (may need to estimate lineset later for other plays)

# #get coordinate values for each player at the lineset and ball_snap
# tracking_line_set = tracking[tracking['event'] == 'line_set'].drop(columns=['jerseyNumber','time','x', 'y', 's', 'a', 'dis', 'o', 'dir','playDirection'])
# tracking_ball_snap = tracking[tracking['event'] == 'ball_snap'].drop(columns=['jerseyNumber','time','x', 'y', 's', 'a', 'dis', 'o', 'dir','playDirection'])
                                                

In [110]:
# tracking_line_set.head()

In [112]:
# tracking_line_to_snap = tracking_line_set.merge(tracking_ball_snap, on=['gameId','playId','nflId','displayName','club'],how='left',suffixes=('_ls','_bs'))

In [54]:
#check what players were in motion (can also check shiftSinceLineset later)
df_player_play_motion_ids = df_player_play[df_player_play['motionSinceLineset'] == True][['gameId','playId','nflId']]
#df_player_play_motion_playIds = df_player_play[df_player_play['motionSinceLineset'] == True][['gameId','playId']].drop_duplicates()

Cleaned Motion DataFrame

In [245]:
#create a dataframe with only the players who were labeled as having MotionSinceLineset
tracking_motion = tracking.merge(df_player_play_motion_ids, on = ['gameId','playId','nflId'], how = 'inner')
tracking_motion = tracking_motion.drop(columns=['x', 'y', 's', 'a','dis', 'o', 'dir'])

In [246]:
#calculate distance from player to football
# #MAKE THIS A FUNCTION LATER, NEST IT INTO LARGER ONE

tracking_football = tracking[tracking['displayName'] == 'football'][['gameId','playId','frameId','x_std','y_std']]

tracking_motion = tracking_motion.merge(tracking_football, on = ['gameId','playId','frameId'], suffixes=('','_fb'))

tracking_motion['dis_fb_x'] = tracking_motion['x_std'] - tracking_motion['x_std_fb']
tracking_motion['dis_fb_y'] = tracking_motion['y_std'] - tracking_motion['y_std_fb']

In [277]:
#creates a column of the first frame where the passed event occurred on the specific play
def find_frame_of_event(df, event_name, event_suffix):
    
    #get frameId for every event occurrence per play
    frameIds = df[df['event'] == event_name][['gameId','playId','frameId']].drop_duplicates()

    #get frameId column by merging dataframes
    merged_df = df.merge(frameIds, on=['gameId', 'playId'], suffixes=('', event_suffix))

    return merged_df

#filters the tracking data passed to the frames between 'line_set' and 'ball_snap'
def filter_to_between_line_set_ball_snap(df):

    #filter out any plays where lineset was not marked (ended up removing 0.7% of frames: (1 - len(filtered_tracking_motion) / len(tracking_motion)))
    df_new = (
        df
        .groupby(['gameId', 'playId'])
        .filter(lambda x: (x['event'] == 'line_set').any())
        .reset_index(drop=True)
    )

    #remove frames after ball_snap
    df_new = df_new[df_new['frameType'] != 'AFTER_SNAP']
    
    #add column for frameId when lineset occurred
    filtered_df = find_frame_of_event(df_new, 'line_set', '_ls')
    
    #remove frames before line_snap
    filtered_df = filtered_df[filtered_df['frameId'] >= filtered_df['frameId_ls']]

    #add column for frameId when the ball was snapped
    filtered_df = find_frame_of_event(filtered_df, 'ball_snap', '_bs')

    return filtered_df

In [319]:
tracking_motion_filtered = filter_to_between_line_set_ball_snap(tracking_motion)

In [321]:
tracking_motion_filtered.head()

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,...,dir_std,s_std,a_std,dis_std,x_std_fb,y_std_fb,dis_fb_x,dis_fb_y,frameId_ls,frameId_bs
0,2022091200,64,46189.0,Will Dissly,43,BEFORE_SNAP,2022-09-13 00:16:07.7,89.0,SEA,right,...,34068,6,17,1,3949,2394,-477,252,43,114
1,2022091200,64,46189.0,Will Dissly,44,BEFORE_SNAP,2022-09-13 00:16:07.8,89.0,SEA,right,...,33651,5,11,0,3949,2394,-477,252,43,114
2,2022091200,64,46189.0,Will Dissly,45,BEFORE_SNAP,2022-09-13 00:16:07.9,89.0,SEA,right,...,35166,8,17,1,3949,2394,-477,252,43,114
3,2022091200,64,46189.0,Will Dissly,46,BEFORE_SNAP,2022-09-13 00:16:08,89.0,SEA,right,...,34832,9,12,1,3949,2395,-475,251,43,114
4,2022091200,64,46189.0,Will Dissly,47,BEFORE_SNAP,2022-09-13 00:16:08.1,89.0,SEA,right,...,34926,11,12,1,3949,2395,-474,250,43,114


things to check
- distance from ball start to finish
- net x,y distance
- Time moving before snap
- Change in direction
- When did they start the motion/finish the motion (will need to make some parameter of change in distance)

Starting to add columns that could be useful

In [325]:
#drop unnecessary columns (should probably drop some of these earlier)
tracking_simple = tracking_motion_filtered.drop(columns=['frameType','time', 'jerseyNumber','playDirection','event'])

In [334]:
#frames before snap (note: a frame is .1 second, adjust for anything in the final write up)
tracking_simple['frames_bf_snap'] = tracking_simple['frameId_bs'] - tracking_simple['frameId']

#player's distance from football (x and y)
tracking_simple['x_dis_from_fb'] = tracking_simple['x_std'] - tracking_simple['x_std']
tracking_simple['y_dis_from_fb'] = tracking_simple['y_std'] - tracking_simple['y_std']

In [352]:
#get initial coordinates
tracking_simple.loc[tracking_simple['frameId'] == tracking_simple['frameId_ls'], 'initial_x_std'] = tracking_simple['x_std']
tracking_simple['initial_x_std'] = tracking_simple['initial_x_std'].ffill().astype(int)

tracking_simple.loc[tracking_simple['frameId'] == tracking_simple['frameId_ls'], 'initial_y_std'] = tracking_simple['y_std']
tracking_simple['initial_y_std'] = tracking_simple['initial_y_std'].ffill().astype(int)

#find relative change in distance since initial frame
#NOTE: at some point will need to figure out how to factor for positive vs negative
tracking_simple['x_change_all'] = tracking_simple['initial_x_std'] - tracking_simple['x_std']
tracking_simple['y_change_all'] = tracking_simple['initial_y_std'] - tracking_simple['y_std']

In [354]:
tracking_simple.head()

,gameId,playId,nflId,displayName,frameId,club,x_std,y_std,o_std,dir_std,...,y_std_fb,dis_fb_x,dis_fb_y,frameId_ls,frameId_bs,frames_bf_snap,x_dis_from_fb,y_dis_from_fb,initial_x_std,initial_y_std
0,2022091200,64,46189.0,Will Dissly,43,SEA,3472,2646,443,34068,...,2394,-477,252,43,114,71,0,0,3472,2646
1,2022091200,64,46189.0,Will Dissly,44,SEA,3472,2646,565,33651,...,2394,-477,252,43,114,70,0,0,3472,2646
2,2022091200,64,46189.0,Will Dissly,45,SEA,3472,2646,709,35166,...,2394,-477,252,43,114,69,0,0,3472,2646
3,2022091200,64,46189.0,Will Dissly,46,SEA,3474,2646,831,34832,...,2395,-475,251,43,114,68,0,0,3472,2646
4,2022091200,64,46189.0,Will Dissly,47,SEA,3475,2645,1172,34926,...,2395,-474,250,43,114,67,0,0,3472,2646
